### Potential Errors:
1) Biomass loaded effluent liquid has assumed density of 1000 g/L <br>
2) Acetate as a liquid product not considered

In [1]:
import pandas as pd
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)

In [2]:
# Primary inputs from user
run_id = 'CST' + str(input("Enter the digits from the run ID (CST'XXX'). Only enter the digits!: "))
start_time = float(input("Enter the START time. If timepoint is not in raw data, the next timepoint will be used. "))
end_time = float(input("Enter the END time. If timepoint is not in raw data, the previous timepoint will be used. "))

# Input and output file strings
infile = '../CST Data Files/run_set_data_' + run_id + '.csv'
outfile = '../DEV Blend Mass Balance Run List PYTHON v0.5.xlsx'
varfile = 'FDI Tags (do not touch)/tank_tags_map.csv'

Enter the digits from the run ID (CST'XXX'). Only enter the digits!: 310
Enter the START time. If timepoint is not in raw data, the next timepoint will be used. 552
Enter the END time. If timepoint is not in raw data, the previous timepoint will be used. 614


In [3]:
# Constants and assumptions (all begin with _)
_OD_to_gDCW_per_L = 0.300
_T_ref_K = 298.0
_P_ref_bar = 1.0
_R__L_bar_per_K_mol = 0.08314
_working_mass_g = 425.0
_culture_density_g_per_L = 1000.0
_working_vol_L = _working_mass_g / _culture_density_g_per_L
_Biomass_MW = 23.19
_BDO_MW = 90.14

# dict to hold constants pertaining to each element (C H O N)
_elements = {'C': {'molar_mass':12.0, 
                   'mass_fraction_in':{'H2O':0.0, 'biomass':0.5424, 'BDO':0.5333},
                   'moles_in':        {'argon':0,'carbon_dioxide':1,'ethane':2,'hydrogen':0,'methane':1,'nitrogen':0,'oxygen':0,'propane':3}},
             'H': {'molar_mass':1.0, 
                   'mass_fraction_in':{'H2O':0.1111, 'biomass':0.0714, 'BDO':0.1111},
                   'moles_in':        {'argon':0,'carbon_dioxide':0,'ethane':6,'hydrogen':2,'methane':4,'nitrogen':0,'oxygen':0,'propane':8}},
             'O': {'molar_mass':16.0, 
                   'mass_fraction_in':{'H2O':0.8889, 'biomass':0.1778, 'BDO':0.3556},
                   'moles_in':        {'argon':0,'carbon_dioxide':2,'ethane':0,'hydrogen':0,'methane':0,'nitrogen':0,'oxygen':2,'propane':0}},
             'N': {'molar_mass':14.0, 
                   'mass_fraction_in':{'H2O':0.0, 'biomass':0.2084, 'BDO':0.0},
                   'moles_in':        {'argon':0,'carbon_dioxide':0,'ethane':0,'hydrogen':0,'methane':0,'nitrogen':2,'oxygen':0,'propane':0}},
            }

# Large dict-based data structure to be populated with data for each of 8 components
components = {'argon':         {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'carbon_dioxide':{'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'ethane':        {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'hydrogen':      {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'methane':       {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'nitrogen':      {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'oxygen':        {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None},
              'propane':       {'x_in_raw':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_in_norm':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'F_in_mph':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}, 
                                'x_out_raw':None, 
                                'x_out_norm':None, 
                                'F_out_mph':None}
              }

# dict to be populated with flow rates for every in/out gas stream
# volumetric rates populated from averaged raw data, then used to calculate molar rates using PV=nRT
stream_rates = {'sccm/min':{'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None},
                'mph':     {'BLEND':None, 'AIR':None, 'CH4':None, 'CNG':None, 'N2':None, 'O2':None}
               }

# Calculated total inlet/effluent rates using NORMALIZED compositions (inlet is NOT simply summed from stream_rates)
total_inlet_norm_mph = 0.0
total_effluent_norm_mph = 0.0

# dict to be populated with elemental mass flow rates: gas in/out, liquid in/out, and accumulation, all in g/hr 
elem_rates = {'C':{'inlet_gph':0.0, 
                   'effluent_gph':0.0,
                   'liquid_gph':None,
                   'draw_gph':None,
                   'accumulation_gph':None},
              'H':{'inlet_gph':0.0, 
                   'effluent_gph':0.0,
                   'liquid_gph':None,
                   'draw_gph':None,
                   'accumulation_gph':None},
              'O':{'inlet_gph':0.0, 
                   'effluent_gph':0.0,
                   'liquid_gph':None,
                   'draw_gph':None,
                   'accumulation_gph':None},
              'N':{'inlet_gph':0.0, 
                   'effluent_gph':0.0,
                   'liquid_gph':None,
                   'draw_gph':None,
                   'accumulation_gph':None},
             }

In [ ]:
# Take user input to determine which streams will be considered in calculations
used_streams = []

answer = input("Are you using the BLEND gas stream? Enter 'y' or 'n': ")
if answer == 'y':
    used_streams.append('BLEND')
answer = input("Are you using other gas streams? Enter 'y' or 'n': ")
if answer == 'y':
    for stream in ['AIR','CH4','CNG','N2','O2']:
        answer = input(f"Are you using the {stream} stream? Enter 'y' or 'n': ")
        if answer == 'y':
            used_streams.append(stream)

print(f"Gas streams considered in calculations: {used_streams}")

In [ ]:
# Load data into DataFrame, and Trim DF to only include data within the specified time window. 
# Set index to ferm age (time), and don't drop column!
# If exact input start_time/end_time don't exist, next/previous timepoints will be picked instead
# Remove 'NaN' values from raw data
raw_df = pd.read_csv(infile)
raw_df = raw_df[raw_df['fermentation_age_hours'] >= start_time]
raw_df = raw_df[raw_df['fermentation_age_hours'] <= end_time]
raw_df = raw_df.set_index('fermentation_age_hours', drop=False)
raw_df.fillna(0.0, inplace=True)
# raw_df = raw_df[raw_df['offgas2.rms_flow'] >= 50.0]

# Pull tank number from raw data as a string, e.g. "02" from "T02"
tank = raw_df.at[raw_df.index[0],'fermentor'][1:3]

# Initialize dict with keys only (values = None)
tag_dict = dict.fromkeys(['BDO, mg/L', 'OD, raw', 'Feed Scale, kg', 
                          'BLEND_F_in', 'BLEND_x_argon_in_raw', 'BLEND_x_carbon_dioxide_in_raw', 'BLEND_x_ethane_in_raw', 'BLEND_x_hydrogen_in_raw', 'BLEND_x_methane_in_raw', 'BLEND_x_nitrogen_in_raw', 'BLEND_x_oxygen_in_raw', 'BLEND_x_propane_in_raw',
                          'AIR_F_in', 'AIR_x_argon_in_raw', 'AIR_x_carbon_dioxide_in_raw', 'AIR_x_ethane_in_raw', 'AIR_x_hydrogen_in_raw', 'AIR_x_methane_in_raw', 'AIR_x_nitrogen_in_raw', 'AIR_x_oxygen_in_raw', 'AIR_x_propane_in_raw',
                          'CH4_F_in', 'CH4_x_argon_in_raw', 'CH4_x_carbon_dioxide_in_raw', 'CH4_x_ethane_in_raw', 'CH4_x_hydrogen_in_raw', 'CH4_x_methane_in_raw', 'CH4_x_nitrogen_in_raw', 'CH4_x_oxygen_in_raw', 'CH4_x_propane_in_raw',
                          'CNG_F_in', 'CNG_x_argon_in_raw', 'CNG_x_carbon_dioxide_in_raw', 'CNG_x_ethane_in_raw', 'CNG_x_hydrogen_in_raw', 'CNG_x_methane_in_raw', 'CNG_x_nitrogen_in_raw', 'CNG_x_oxygen_in_raw', 'CNG_x_propane_in_raw',
                          'O2_F_in', 'O2_x_argon_in_raw', 'O2_x_carbon_dioxide_in_raw', 'O2_x_ethane_in_raw', 'O2_x_hydrogen_in_raw', 'O2_x_methane_in_raw', 'O2_x_nitrogen_in_raw', 'O2_x_oxygen_in_raw', 'O2_x_propane_in_raw',
                          'N2_F_in', 'N2_x_argon_in_raw', 'N2_x_carbon_dioxide_in_raw', 'N2_x_ethane_in_raw', 'N2_x_hydrogen_in_raw', 'N2_x_methane_in_raw', 'N2_x_nitrogen_in_raw', 'N2_x_oxygen_in_raw', 'N2_x_propane_in_raw',
                          'x_argon_out_raw', 'x_carbon_dioxide_out_raw', 'x_ethane_out_raw', 'x_hydrogen_out_raw', 'x_methane_out_raw', 'x_nitrogen_out_raw', 'x_oxygen_out_raw', 'x_propane_out_raw'
])

# Fill in dict values with header tags for input tank, from varfile csv
tag_map_df = pd.read_csv(varfile, index_col='Tank')
for key in tag_dict.keys():
    tag_dict[key] = tag_map_df.at[int(tank), key]

start_time = raw_df['fermentation_age_hours'].min()
end_time = raw_df['fermentation_age_hours'].max()

# Warn if steady state is not reached. Steady state being +/- 25% of BDO, 2% of OD
bdo_start = raw_df.at[start_time, tag_dict['BDO, mg/L']]
bdo_end = raw_df.at[end_time, tag_dict['BDO, mg/L']]
od_start = raw_df.at[start_time, tag_dict['OD, raw']]
od_end = raw_df.at[end_time, tag_dict['OD, raw']]

print("-----------------------------------------------------")
print('Run Number:'.ljust(30) + f'{run_id}')
print('Tank:'.ljust(30) + f'T{tank}')
print('Start time:'.ljust(30) + f'{start_time}')
print('End time:'.ljust(30) + f'{end_time}')
print('Initial BDO in mg/L:'.ljust(30) + f'{bdo_start}')
print('Final BDO in mg/L'.ljust(30) + f'{bdo_end}')
print('Initial OD:'.ljust(30) + f'{od_start}')
print('Final OD:'.ljust(30) + f'{od_end}')

In [ ]:
# Average raw data over interval for BDO, OD, and (8 component mass fractions) * (6 possible inlet + 1 outlet) streams
# Load into 'components' data structure
BDO_mg_per_L = raw_df[tag_dict['BDO, mg/L']].mean()
OD_raw = raw_df[tag_dict['OD, raw']].mean()

for comp,c in components.items():
    c['x_out_raw'] = raw_df[tag_dict[f'x_{comp}_out_raw']].mean()
    for s in used_streams:
        c['x_in_raw'][s] = raw_df[tag_dict[f'{s}_x_{comp}_in_raw']].mean()

for s in used_streams:
    stream_rates['sccm/min'][s] = raw_df[tag_dict[f'{s}_F_in']].mean()

In [ ]:
## Inlet gas

# Calculate comp_fraction_total
# 8 components * # of used streams - find sum of all component fractions (averaged over time period, given in % points).
# Should be close to 100.0 * # of streams (negligible components are ignored)
comp_fraction_total = 0.0
for comp,c in components.items():
    for s in used_streams:
        comp_fraction_total += c['x_in_raw'][s]

# With comp_fraction_total, calculate normalized compositions so that each of # of streams x 8 component fractions add up to 100 % points
for comp,c in components.items():
    for s in used_streams:
        c['x_in_norm'][s] = c['x_in_raw'][s] * len(used_streams) * 100.0 / comp_fraction_total

# Convert total volumetric flow rate through each used stream (SCCM/min = standard cubic centimeters/min; same as mL/min)
# to total molar flow rates (mph) using PV=nRT.
for s in used_streams:
    stream_rates['mph'][s] = stream_rates['sccm/min'][s] * (1/1000) * 60 * _P_ref_bar / (_R__L_bar_per_K_mol * _T_ref_K)  
    
# Calculate molar flow rates for each comp in each stream = stream flow rate * NORMALIZED component fraction / 100%
# Also sum up to get total inlet rate
for comp,c in components.items():
    for s in used_streams:
        c['F_in_mph'][s] = stream_rates['mph'][s] * c['x_in_norm'][s] / 100.0
        total_inlet_norm_mph += c['F_in_mph'][s]
        
        # Calculate elemental molar rate for C H O N; sum to get total
        # = comp molar rate * mol of elem in mol of comp * elem molar mass (g/mol)
        for elem, e in elem_rates.items():
            e['inlet_gph'] += c['F_in_mph'][s] * _elements[elem]['moles_in'][comp] * _elements[elem]['molar_mass']

In [ ]:
## Effluent Gas - same steps as for Inlet Gas, but only one out stream; also need to calculate total F_out

comp_fraction_total = 0.0
for comp,c in components.items():
    comp_fraction_total += c['x_out_raw']

for comp,c in components.items():
    c['x_out_norm'] = c['x_out_raw'] * 100.0 / comp_fraction_total

# Need to calculate total effluent molar flow rate using argon mass balance: Argon mph in = Argon mph out
# (total F_in_mph) * (Argon fraction in) = (total F_out_mph) * (Argon fraction out)
# (total F_in_mph) * (Argon fraction in) = (total Argon F_in) = (sum(stream F_in_mph * stream Argon fraction))
# total F_out_mph = (sum(stream F_in_mph * stream Argon fraction)) / (Argon fraction out)
total_argon_in_molph = 0.0
for s in used_streams:
    total_argon_in_molph += stream_rates['mph'][s] * components['argon']['x_in_norm'][s]
    
total_effluent_norm_mph = total_argon_in_molph / components['argon']['x_out_norm']

for comp,c in components.items():
    c['F_out_mph'] = total_effluent_norm_mph * c['x_out_norm'] / 100.0

    # Calculate elemental molar rate for C H O N; sum to get total
    # = comp molar rate * mol of elem in mol of comp * elem molar mass (g/mol)
    for elem, e in elem_rates.items():
        e['effluent_gph'] += c['F_out_mph'] * _elements[elem]['moles_in'][comp] * _elements[elem]['molar_mass']

In [ ]:
## Liquid Feed
# Liquid feed rate is d(feed scale)/dt. This mass balance is designed for a single dilution rate (feed rate / V)
t_start = start_time
t_start_plus1 = start_time + 1.0
t_end = end_time
t_end_minus1 = end_time - 1.0

# Feed rate = (feed scale reading @ t - feed scale reading @ t+1) / 1 hour
feed_rate_start = raw_df.at[t_start, tag_dict['Feed Scale, kg']] - raw_df.at[t_start_plus1, tag_dict['Feed Scale, kg']]
feed_rate_end = raw_df.at[t_end_minus1, tag_dict['Feed Scale, kg']] - raw_df.at[t_end, tag_dict['Feed Scale, kg']]

# Conditional statement in case we don't have feed scale data, take user input for dilution rate
if feed_rate_end == 0.0:
    dilution_rate = float(input("Feed scale data is missing. Please enter your assumed dilution rate, e.g. '0.03': "))
    liq_feed_rate_gph = dilution_rate * _working_mass_g
else:
    # scales all should be reading in kg units => convert to g
    liq_feed_rate_gph = feed_rate_end * 1000.00
    dilution_rate = liq_feed_rate_gph / _working_mass_g
    
for elem, e in elem_rates.items():
    e['liquid_gph'] = liq_feed_rate_gph * _elements[elem]['mass_fraction_in']['H2O']

In [ ]:
## Product Accumulation

# Calculate accumulation (g/L) of liquid products (BDO, BioMass, H2O)
# BDO given in mg/L, convert to g/L
# accumulation of H2O = -(accumulation of BDO and BM)
delta_BDO_gpL = (bdo_end - bdo_start) / 1000.0
delta_BM_gpL = (od_end - od_end) * _OD_to_gDCW_per_L
delta_H2O_gpL = -(delta_BDO_gpL + delta_BM_gpL)

# Start with Accumulation = Mass * (delta_BDO * sigma_O_BDO + delta_BM * sigma_O_BM + delta_H2O * sigma_O_H2O)
for elem, e in elem_rates.items():
    e['accumulation_gph'] = _working_vol_L * (delta_BDO_gpL * _elements[elem]['mass_fraction_in']['BDO'] + \
                                              delta_BM_gpL * _elements[elem]['mass_fraction_in']['biomass'] + \
                                              delta_H2O_gpL * _elements[elem]['mass_fraction_in']['H2O']) \
                            / (end_time - start_time)

In [ ]:
## Liquid Draw

# Determine average liquid mass fractions of [BDO, BM, H2O] in liquid draw/effluent. Assume 1000.0 g/L density.
w_BDO_gpL = BDO_mg_per_L / 1000
w_BM_gpL = OD_raw * _OD_to_gDCW_per_L
w_H2O_gpL = 1000.0 - w_BDO_gpL - w_BM_gpL

# Calculate total draw rate in Lph from two oxygen balances:
# 1: O_draw_gph = O_inlet_gph + O_liquid_gph - O_effluent_gph - O_acc 
# 2: O_draw_gph = draw_rate_Lph * (w_BDO_liq * w_O_BDO + w_BM_liq * w_O_BDO + w_H2O_liq * w_O_H2O)
# Another option is to simply set 'Draw Rate, g/hr' = 'Liquid Feed Rate, g/hr'
O = elem_rates['O']
O_mf = _elements['O']['mass_fraction_in']

draw_rate_Lph = (O['liquid_gph'] + O['inlet_gph'] - O['effluent_gph'] - O['accumulation_gph']) \
    / (w_BDO_gpL * O_mf['BDO'] + w_BM_gpL * O_mf['biomass'] + w_H2O_gpL * O_mf['H2O'])

for elem, e in elem_rates.items():
    elem_mf = _elements[elem]['mass_fraction_in']
    e['draw_gph'] = draw_rate_Lph * (w_BDO_gpL * elem_mf['BDO'] + w_BM_gpL * elem_mf['biomass'] + w_H2O_gpL * elem_mf['H2O'] )

In [ ]:
draw_rate_Lph * 1000

In [ ]:
liq_feed_rate_gph

In [ ]:
## CHON Closures

closures = {'C':None, 'H':None, 'O':None, 'N':None}

for elem in closures:
    e = elem_rates[elem]
    closures[elem] = (e['effluent_gph'] + e['draw_gph'] + e['accumulation_gph']) \
                   / (e['inlet_gph'] + e['liquid_gph'])
    
print(f"Carbon Closure: {closures['C']}")

In [ ]:
# Calculate KPI rates

KPI_rates = {'carbon_dioxide':{'mmph':None},
             'ethane':        {'mmph':None}, 
             'methane':       {'mmph':None},
             'nitrogen':      {'mmph':None},
             'oxygen':        {'mmph':None}
            }

# CER = out - in, XUR = in - out; (1000 mmol/mol) rate conversion
for comp, r in KPI_rates.items():
    c = components[comp]
    # total comp mph in = sum of component mph through each stream. Need to filter out None values first.
    c_F_in_mph = sum(filter(None, c['F_in_mph'].values()))
    if comp == 'carbon_dioxide':
        r['mmph'] = (c['F_out_mph'] - c_F_in_mph) * 1000.0
    else:
        r['mmph'] = (c_F_in_mph - c['F_out_mph']) * 1000.0

# Biomass ER: OD * (g DCW / L / OD) * L * (1/hr) / (g DCW / mol DCW) * (1000 mmol DCW / mol DCW) = mmol DCW / hr
biomass_ER = OD_raw * _OD_to_gDCW_per_L * _working_vol_L * dilution_rate / _Biomass_MW  * 1000

# 23BDO ER: (mg BDO / L) * L * (1/hr) / (g BDO / mol DCW) * (1000 mmol BDO / mol BDO) * (g BDO / 1000 mg BDO) = mmol BDO / hr
BDO_ER = BDO_mg_per_L * _working_vol_L * dilution_rate / _BDO_MW

In [16]:
KPI_rates

{'carbon_dioxide': {'mmph': None},
 'ethane': {'mmph': None},
 'methane': {'mmph': None},
 'nitrogen': {'mmph': None},
 'oxygen': {'mmph': None}}

In [ ]:
print(biomass_ER)
print(BDO_ER)

In [ ]:
# Read current run list into DataFrame. Append new empty row, and set idx to integer index of this last row
results = pd.read_excel(outfile, index_col=0)
results = results.append(pd.Series(), ignore_index=True)
idx = results.index[-1]

results.at[idx, 'Run_ID'] = run_id
results.at[idx, 'Tank'] = int(tank)
results.at[idx, 'DCW, g/L/OD'] = _OD_to_gDCW_per_L
results.at[idx, 'Assumed Working Vol, L'] = _working_mass_g / 1000

# If exact input start_time/end_time don't exist, next/previous timepoints will be picked instead
results.at[idx, 'Start Time, hours'] = raw_df['fermentation_age_hours'].min()
results.at[idx, 'End Time, hours'] = raw_df['fermentation_age_hours'].max()

results.at[idx, 'Initial BDO, mg/L'] = bdo_start
results.at[idx, 'Final BDO, mg/L'] = bdo_end
results.at[idx, 'Initial OD, raw'] = od_start
results.at[idx, 'Final OD, raw'] = od_end

results.at[idx, 'Total Inlet Gas Rate, mol/hr'] = total_inlet_mph

results.at[idx, 'Inlet C Rate, g/hr'] = in_gas_C_rate_mph * 12.0
results.at[idx, 'Inlet H Rate, g/hr'] = in_gas_H_rate_mph * 1.0
results.at[idx, 'Inlet O Rate, g/hr'] = in_gas_O_rate_mph * 16.0
results.at[idx, 'Inlet N Rate, g/hr'] = in_gas_N_rate_mph * 14.0

results.at[idx, 'Effluent C Rate, g/hr'] = out_gas_C_rate_mph * 12.0
results.at[idx, 'Effluent H Rate, g/hr'] = out_gas_H_rate_mph * 1.0
results.at[idx, 'Effluent O Rate, g/hr'] = out_gas_O_rate_mph * 16.0
results.at[idx, 'Effluent N Rate, g/hr'] = out_gas_N_rate_mph * 14.0

results.at[idx, 'Liquid Feed Rate, g/hr'] = liq_feed_rate_gph

results.at[idx, 'Dilution Rate, hrs^-1'] = dilution_rate

results.at[idx, 'Liquid C Rate, g/hr'] = results.at[idx, 'Liquid Feed Rate, g/hr'] * _water_mass_fraction[0]
results.at[idx, 'Liquid H Rate, g/hr'] = results.at[idx, 'Liquid Feed Rate, g/hr'] * _water_mass_fraction[1]
results.at[idx, 'Liquid O Rate, g/hr'] = results.at[idx, 'Liquid Feed Rate, g/hr'] * _water_mass_fraction[2]
results.at[idx, 'Liquid N Rate, g/hr'] = results.at[idx, 'Liquid Feed Rate, g/hr'] * _water_mass_fraction[3]

results.at[idx, 'Accumulation C, g/hr'] = (_working_mass_g / 1000.0 ) * ( delta_w_bdo_gpL * _BDO_mass_fraction[0] + \
                                                                    delta_w_bm_gpL * _biomass_mass_fraction[0] + \
                                                                    delta_w_h2o_gpL * _water_mass_fraction[0] ) \
                            / (end_time - start_time)
results.at[idx, 'Accumulation H, g/hr'] = (_working_mass_g / 1000.0 ) * ( delta_w_bdo_gpL * _BDO_mass_fraction[1] + \
                                                                            delta_w_bm_gpL * _biomass_mass_fraction[1] + \
                                                                            delta_w_h2o_gpL * _water_mass_fraction[1] ) \
                                    / (end_time - start_time)
results.at[idx, 'Accumulation O, g/hr'] = (_working_mass_g / 1000.0 ) * ( delta_w_bdo_gpL * _BDO_mass_fraction[2] + \
                                                                            delta_w_bm_gpL * _biomass_mass_fraction[2] + \
                                                                            delta_w_h2o_gpL * _water_mass_fraction[2] ) \
                                    / (end_time - start_time)
results.at[idx, 'Accumulation N, g/hr'] = (_working_mass_g / 1000.0 ) * ( delta_w_bdo_gpL * _BDO_mass_fraction[3] + \
                                                                            delta_w_bm_gpL * _biomass_mass_fraction[3] + \
                                                                            delta_w_h2o_gpL * _water_mass_fraction[3] ) \
                                    / (end_time - start_time)

results.at[idx, 'w_BDO_liq_g/L'] = results.at[idx, 'BDO, mg/L'] / 1000.0
results.at[idx, 'w_BM_liq_g/L'] = results.at[idx, 'OD, raw'] * _OD_to_gDCW_per_L
results.at[idx, 'w_H2O_liq_g/L'] = 1000.0 - results.at[idx, 'w_BDO_liq_g/L'] - results.at[idx, 'w_BM_liq_g/L']

results.at[idx, 'Draw Rate, L/hr'] = (results.at[idx, 'Liquid O Rate, g/hr'] + results.at[idx, 'Inlet O Rate, g/hr'] - results.at[idx, 'Effluent O Rate, g/hr'] - results.at[idx, 'Accumulation O, g/hr']) \
    / (results.at[idx, 'w_BDO_liq_g/L'] * _BDO_mass_fraction[2] + results.at[idx, 'w_BM_liq_g/L'] * _biomass_mass_fraction[2] + results.at[idx, 'w_H2O_liq_g/L'] * _water_mass_fraction[2])
results.at[idx, 'Draw Rate, g/hr'] = results.at[idx, 'Draw Rate, L/hr'] *1000.0

results.at[idx, 'Draw C Rate, g/hr'] = results.at[idx, 'Draw Rate, L/hr'] * (results.at[idx, 'w_BDO_liq_g/L'] * _BDO_mass_fraction[0] + \
                                                                                    results.at[idx, 'w_BM_liq_g/L'] * _biomass_mass_fraction[0] + \
                                                                                    results.at[idx, 'w_H2O_liq_g/L'] * _water_mass_fraction[0])
results.at[idx, 'Draw H Rate, g/hr'] = results.at[idx, 'Draw Rate, L/hr'] * (results.at[idx, 'w_BDO_liq_g/L'] * _BDO_mass_fraction[1] + \
                                                                                    results.at[idx, 'w_BM_liq_g/L'] * _biomass_mass_fraction[1] + \
                                                                                    results.at[idx, 'w_H2O_liq_g/L'] * _water_mass_fraction[1])
results.at[idx, 'Draw O Rate, g/hr'] = results.at[idx, 'Draw Rate, L/hr'] * (results.at[idx, 'w_BDO_liq_g/L'] * _BDO_mass_fraction[2] + \
                                                                                    results.at[idx, 'w_BM_liq_g/L'] * _biomass_mass_fraction[2] + \
                                                                                    results.at[idx, 'w_H2O_liq_g/L'] * _water_mass_fraction[2])
results.at[idx, 'Draw N Rate, g/hr'] = results.at[idx, 'Draw Rate, L/hr'] * (results.at[idx, 'w_BDO_liq_g/L'] * _BDO_mass_fraction[3] + \
                                                                                    results.at[idx, 'w_BM_liq_g/L'] * _biomass_mass_fraction[3] + \
                                                                                    results.at[idx, 'w_H2O_liq_g/L'] * _water_mass_fraction[3])

results.at[idx, 'C Closure'] = ( results.at[idx, 'Effluent C Rate, g/hr'] + results.at[idx, 'Draw C Rate, g/hr'] + results.at[idx, 'Accumulation C, g/hr'] ) /\
                                ( results.at[idx, 'Inlet C Rate, g/hr'] + results.at[idx, 'Liquid C Rate, g/hr'] )
results.at[idx, 'H Closure'] = ( results.at[idx, 'Effluent H Rate, g/hr'] + results.at[idx, 'Draw H Rate, g/hr'] + results.at[idx, 'Accumulation H, g/hr'] ) /\
                                ( results.at[idx, 'Inlet H Rate, g/hr'] + results.at[idx, 'Liquid H Rate, g/hr'] )
results.at[idx, 'O Closure'] = ( results.at[idx, 'Effluent O Rate, g/hr'] + results.at[idx, 'Draw O Rate, g/hr'] + results.at[idx, 'Accumulation O, g/hr'] ) /\
                                ( results.at[idx, 'Inlet O Rate, g/hr'] + results.at[idx, 'Liquid O Rate, g/hr'] )
results.at[idx, 'N Closure'] = ( results.at[idx, 'Effluent N Rate, g/hr'] + results.at[idx, 'Draw N Rate, g/hr'] + results.at[idx, 'Accumulation N, g/hr'] ) /\
                                ( results.at[idx, 'Inlet N Rate, g/hr'] + results.at[idx, 'Liquid N Rate, g/hr'] )

results.at[idx, 'Used streams'] = used_streams

if KPI['rate'] == 'CER':
    results.at[idx, KPI_with_units] = (results.at[idx, tag_out] - sum_F_in_molph) * 1000
else:
    results.at[idx, KPI_with_units] = (sum_F_in_molph - results.at[idx, tag_out]) * 1000

results.at[idx, 'Biomass ER, mmol/hr'] = results.at[idx, 'OD, raw'] * _OD_to_gDCW_per_L * \
                                    results.at[idx, 'Assumed Working Vol, L'] / Biomass_MW * \
                                    results.at[idx, 'Dilution Rate, hrs^-1'] * 1000

results.at[idx, '2,3-BDO, mmol/hr'] = results.at[idx, 'BDO, mg/L'] * \
                                        results.at[idx, 'Assumed Working Vol, L'] / _23BDO_MW * \
                                        results.at[idx, 'Dilution Rate, hrs^-1']

In [ ]:
# Re-organize headers so that Closure and KPI columns come first
beginning_cols = ['Run_ID', 'Tank', 'Start Time, hours', 'End Time, hours', 
                'C Closure', 'H Closure', 'O Closure', 'N Closure', 
                'CER, mmol/hr', 'EUR, mmol/hr', 'MUR, mmol/hr', 'NUR, mmol/hr', 'OUR, mmol/hr', 
                'Biomass ER, mmol/hr', '2,3-BDO, mmol/hr', 'DCW, g/L/OD', 'Assumed Working Vol, L'
]

# Pull just this row, with re-organized cols, to append to outfile
reorganized_df = results.loc[[idx], [col for col in beginning_cols] + [col for col in results if col not in beginning_cols]]

In [ ]:
# Exception handling for PermissionError (e.g. if someone has the output file open)
try:
    # Append to first unoccupied row of 'Python Raw' worksheet in outfile
    with pd.ExcelWriter(outfile, engine='openpyxl', mode='a') as writer:
        book = load_workbook(outfile)
        writer.book = book
        writer.sheets = {ws.title: ws for ws in book.worksheets}
        reorganized_df.to_excel(writer, sheet_name='Python Raw', startrow=writer.sheets['Python Raw'].max_row, index=True, header=False)
except PermissionError as permission_error:
    print("-----------------------------------------------------")
    print(permission_error)
    print("Please close the output file and try again!")